# IBM Data Science Specialization Capstone
This notebook is part of the IBM datascience capstone projects.

## Segmenting and Clustering Neighborhoods in Toronto - Part 1

In this notebook, we will collect the Toronto neighborhoods data from a web page, convert it to a pandas dataframe, and clean and pre-process it.

### Install required packages

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install requests

### Import dependencies

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Collect neighborhood data for Toronto

The data will be collected from Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This page contains a list of postal codes in Canada where the first letter is M.

Get the HTML text from the web page using requests module.

In [3]:
# Get the HTML text from the wiki page
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_text = requests.get(wiki_url).text

Use BeautifulSoup module to extract the table data from the HTML text.

In [4]:
# Extract the table data using BeautifulSoup
soup = BeautifulSoup(html_text)
table = soup.find('table', attrs={'class':'wikitable sortable'})
trs = table.find_all('tr')
print("Number of rows in the table(including columns header): ",
      len(trs))

# Extract the text from all the table cells and add all rows
# to a list of rows.
rows = list()
for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        # Ignore empty rows with no 'td',
        # applicable for the column headers row.
        rows.append(row)

print("Number of rows with data in the table: ", len(rows))


Number of rows in the table(including columns header):  289
Number of rows with data in the table:  288


## Create the pandas dataframe for neighborhood data

Convert the data to a pandas dataframe with columns 'PostalCode', 'Borough', and 'Neighborhood'.

In [5]:
df = pd.DataFrame(rows,
                  columns=['PostalCode', 'Borough', 'Neighborhood'])
print("Dataframe shape: ", df.shape)
df.head(10)


Dataframe shape:  (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Remove rows with column 'Not assigned' for column 'Borough'.

In [6]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
print("Dataframe shape: ", df.shape)
df.head(10)


Dataframe shape:  (211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
So, replace 'Neighborhood' columns with value as 'Not assigned' with the value of its 'Borough'.

In [7]:
df['Neighborhood'] = df.apply(
    lambda row: 
    row['Borough'] if row['Neighborhood'] == 'Not assigned' 
    else row['Neighborhood'],
    axis=1)
print("Dataframe shape: ", df.shape)
df.head(10)


Dataframe shape:  (211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. Combine rows with the same postal code into a single row.
For example, in the table on the Wikipedia page, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [8]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)


In [9]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
df.shape

(103, 3)